# Skin Segmentation Training in Google Colab

This notebook demonstrates how to use the modular skin segmentation code in Google Colab.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone your GitHub repository
!git clone https://github.com/mjf789/spam_skinsegment.git
%cd spam_skinsegment

In [ ]:
# Install requirements
!pip install -r requirements.txt

In [ ]:
# Import the modules
import sys
sys.path.append('/content/spam_skinsegment')

from src.config import get_colab_config
from src.utils import setup_device
from src.train import Trainer

In [ ]:
# Setup configuration for Colab
config = get_colab_config()

# Optionally modify configuration
config.training.num_epochs = 3  # Test with 3 epochs
config.training.batch_size = 4  # Will be auto-adjusted based on GPU

# Setup device
device = setup_device()

In [ ]:
# Initialize trainer
trainer = Trainer(config, device)

# Optionally resume from checkpoint
# checkpoint_path = "/content/drive/MyDrive/Brand Analysis Study/Models/latest_checkpoint.pth"
# trainer.resume_from_checkpoint(checkpoint_path)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Evaluate on hand-only masks if available
if 'val_hand_only' in trainer.loaders:
    from src.evaluate import eval_iou_per_image
    
    # Load best model
    import torch
    best_checkpoint = torch.load(
        f"{config.checkpoint_dir}/best_model.pth",
        map_location=device
    )
    trainer.model.load_state_dict(best_checkpoint['model_state_dict'])
    
    # Evaluate
    hand_only_iou = eval_iou_per_image(
        trainer.model, 
        trainer.loaders['val_hand_only'], 
        device
    )
    print(f"\nFinal hand-only masks validation mean-IoU: {hand_only_iou:.4f}")
    print(f"Best hand IoU during training: {best_checkpoint['best_hand_iou']:.4f}")